In [5]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy import extract
from sqlalchemy import and_, func

In [6]:
f = "airbnb-listings.csv"
f_df = pd.read_csv(f, delimiter=';', encoding = 'latin-1')
f_df.head()

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,2501888,https://www.airbnb.com/rooms/2501888,20170306202425,2017-03-07,Private bed in home,Sofa bed in the media room on the first floor....,Modernist home with a strong influence from MC...,Sofa bed in the media room on the first floor....,none,Upscale area of West Lake Hills on a bluff ove...,...,10.0,9.0,9.0,NaN,NaN,flexible,1,2.42,"30.2863434604, -97.8104775775","Host Has Profile Pic,Host Identity Verified,Is..."
1,377609,https://www.airbnb.com/rooms/377609,20170306202425,2017-03-07,Lake House minutes to Downtown,NaN,"Located just 7 minutes to Downtown, and 4 minu...","Located just 7 minutes to Downtown, and 4 minu...",none,NaN,...,10.0,10.0,9.0,NaN,NaN,strict,1,0.54,"30.3028051776, -97.7942255855","Host Has Profile Pic,Host Identity Verified,Is..."
2,15643010,https://www.airbnb.com/rooms/15643010,20170306202425,2017-03-07,Modern New Apartment Close to Everything!,"Close to the Barton Creek Greenbelt, S Lamar, ...",NaN,"Close to the Barton Creek Greenbelt, S Lamar, ...",none,NaN,...,10.0,10.0,10.0,NaN,NaN,moderate,1,1.86,"30.2521539737, -97.7995481364","Host Has Profile Pic,Host Identity Verified,Is..."
3,2236988,https://www.airbnb.com/rooms/2236988,20170306202425,2017-03-07,Austin Waterfront Retreat,Magnificent Austin waterfront home on Barton C...,Retreat within the city of Austin. 12 minutes...,Magnificent Austin waterfront home on Barton C...,none,Quiet neighborhood tucked away in Austin. Min...,...,10.0,10.0,10.0,NaN,NaN,strict,1,0.21,"30.2752712576, -97.843114474","Host Has Profile Pic,Is Location Exact"
4,14788276,https://www.airbnb.com/rooms/14788276,20170306202425,2017-03-07,Stay at the Domain's hotest residencial property.,Find indulgence the moment you enter the bouti...,NaN,Find indulgence the moment you enter the bouti...,none,NaN,...,10.0,10.0,9.0,NaN,NaN,moderate,1,0.69,"30.4001335924, -97.7241364202","Host Has Profile Pic,Is Location Exact"


In [7]:
f_df.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

In [8]:
n_df = f_df[[
    'Name','Host Response Rate',
    'City', 'State', 'Zipcode','Country Code', 'Property Type', 
    'Price','Number of Reviews','Review Scores Rating']].copy()
n_df.head()

,Name,Host Response Rate,City,State,Zipcode,Country Code,Property Type,Price,Number of Reviews,Review Scores Rating
0,Private bed in home,100.0,West Lake Hills,TX,78746,US,Other,88.0,88,94.0
1,Lake House minutes to Downtown,100.0,Austin,TX,78746,US,House,185.0,21,96.0
2,Modern New Apartment Close to Everything!,100.0,Austin,TX,78746,US,Apartment,60.0,8,97.0
3,Austin Waterfront Retreat,NaN,Austin,TX,78746,US,House,NaN,5,96.0
4,Stay at the Domain's hotest residencial property.,0.0,Austin,TX,78758,US,Condominium,275.0,3,93.0


In [9]:

n_df.rename(columns={'Zipcode':'Zip',
                     'Country Code':'Country',
                     'Property Type':'Type',
                     'Number of Reviews':'Review Count',
                     'Review Scores Rating':'Ratings'}, 
                 inplace=True)

In [10]:
d = n_df.dropna()

In [11]:
d['State'] = d['State'].str.upper()

In [12]:
d.State.unique()
#

array(['TX', 'MA', 'CO', 'IL', 'NY', 'CA', 'LA', 'TN', 'OR', 'WA', 'MP',
       'DC', 'MD'], dtype=object)

In [13]:
#d = d.set_index('Name')


In [14]:
d['Ratings2'] = d['Ratings'] * (5/100)


In [15]:
d = d.drop(['Ratings'],axis=1)
d.rename(columns={'Ratings2':'Ratings'}, 
                 inplace=True)
d.head()

,Name,Host Response Rate,City,State,Zip,Country,Type,Price,Review Count,Ratings
0,Private bed in home,100.0,West Lake Hills,TX,78746,US,Other,88.0,88,4.70
1,Lake House minutes to Downtown,100.0,Austin,TX,78746,US,House,185.0,21,4.80
2,Modern New Apartment Close to Everything!,100.0,Austin,TX,78746,US,Apartment,60.0,8,4.85
4,Stay at the Domain's hotest residencial property.,0.0,Austin,TX,78758,US,Condominium,275.0,3,4.65
10,Studio Apt Close to Zilker/Barton Springs/360,100.0,Austin,TX,78746,US,Apartment,90.0,45,4.95


In [6]:
database_path = "db6.sqlite"
engine = create_engine(f"sqlite:///resources/{database_path}")
# Query All Records in the the Database
conn = engine
conn.execute('CREATE TABLE airbnb (id integer PRIMARY KEY NOT NULL, Name text,
             Host Response Rate float,
             City text,
             State text,
             Zip integer,
             State text,
             Zip text,
             Country text,
             Type text,
             Price float,
             Review Count integer
             Ratings float);')


SyntaxError: EOL while scanning string literal (<ipython-input-6-4b217ea8485c>, line 5)

In [ ]:
d.to_sql(name='airbnb', con=engine2, if_exists='append', index=False)
data = engine2.execute("SELECT * FROM airbnb")

#for record in data:
 #   print(record)

# API Code # 

In [5]:
engine2 = create_engine("sqlite:///Resources/db4.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine2, reflect=True)
#Inspect engine
inspector = inspect(engine2)


NameError: name 'create_engine' is not defined

In [4]:
Base.classes.keys()

NameError: name 'Base' is not defined